In [1]:
%cd ..

c:\Users\Mirco\Desktop\CV_pw


In [15]:
import cv2
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from _2_Retrival.dataset import YooxDatasetSkinHistogram, YooxDatasetDressHistogram, get_multi_level_hist

from tqdm import tqdm

import os, sys
sys.path.append(os.getcwd())

In [3]:
train_dataset = YooxDatasetDressHistogram()
train_dataset = YooxDatasetSkinHistogram()
train_dataset[0][1].shape
train_dataset[0][0].shape

torch.Size([512])

In [4]:
from sklearn.cluster import KMeans, SpectralClustering
import matplotlib

In [5]:
def palette(img):
    arr = np.asarray(img)
    palette, index = np.unique(asvoid(arr).ravel(), return_inverse=True)
    palette = palette.view(arr.dtype).reshape(-1, arr.shape[-1])
    count = np.bincount(index)
    order = np.argsort(count)
    return palette[order[::-1]]

def asvoid(arr):
    arr = np.ascontiguousarray(arr)
    return arr.view(np.dtype((np.void, arr.dtype.itemsize * arr.shape[-1])))

def hsv(arr, j):
    return matplotlib.colors.rgb_to_hsv(arr)[j]

def pantone(img):
    C = palette(img)#[:50]
    P = None
    n = 0
    for c in C:
        if P is None:
            P = np.array(c[None,:])
        else:
            for d in P:
                if ((c-d)**2).sum() > 50:
                    P = np.concatenate([P,c[None,:]],axis=0)
                    n += 1
                    if n > 50:
                        break

    C = P
    N_COLS = 5
    model = KMeans(N_COLS)
    model.fit(C)
    C = model.cluster_centers_.astype(np.uint8)
    P = np.zeros((200,N_COLS*100,3)).astype(np.uint8)
    #for j in range(3):
    j = 2
    for c in range(N_COLS):
        for d in range(N_COLS):
            #if hsv(C[c], j) > hsv(C[d], j):
            if hsv(C[c], j) > hsv(C[d], j):
                tmp = C[c].copy()
                C[c] = C[d].copy()
                C[d] = tmp.copy()

    for i, c in enumerate(C):
        #P[100*j:100*(1+j),100*i:100*(1+i),:] = c
        P[:,100*i:100*(1+i),:] = c
    return P

In [6]:
img = cv2.imread("dataset/upper_body/images/denim_denim-shirts/12141279ui_0_r.jpg")
img = cv2.imread("dataset/upper_body/images/knitwear_track-tops/46614975wd_0_d.jpg")
img = cv2.imread("dataset/upper_body/images/knitwear_track-tops/46662850ev_0_d.jpg")
C = pantone(img)
cv2.imshow("1",C)
cv2.waitKey()
cv2.destroyAllWindows()

In [7]:
train_loader = DataLoader(
        train_dataset,
        batch_size=64,
        shuffle=True,
        # num_workers=config.NUM_WORKERS,
    )

In [8]:
all_vec = None

for h, img in tqdm(train_loader):
    D =  (h[None,...]**2).sum(2) * (h[:,None,:]**2).sum(2)
    sim = (h[None,...] * h[:,None,:]).sum(2) / np.where(D == 0, 1, D)**0.5 - np.eye(D.shape[0])
    print(sim)
    break

  0%|          | 0/304 [00:04<?, ?it/s]

tensor([[0.0000, 0.6022, 0.6772,  ..., 0.6552, 0.8752, 0.8161],
        [0.6022, 0.0000, 0.4451,  ..., 0.3820, 0.5590, 0.5226],
        [0.6772, 0.4451, 0.0000,  ..., 0.7587, 0.5251, 0.8110],
        ...,
        [0.6552, 0.3820, 0.7587,  ..., 0.0000, 0.4827, 0.7406],
        [0.8752, 0.5590, 0.5251,  ..., 0.4827, 0.0000, 0.7751],
        [0.8161, 0.5226, 0.8110,  ..., 0.7406, 0.7751, 0.0000]],
       dtype=torch.float64)


In [9]:
a_idx = 3
b_idx = sim[a_idx].argmax()

In [10]:
a = img[a_idx]
b = img[b_idx]

cv2.imshow("a", a.numpy())
cv2.imshow("b", b.numpy())

cv2.waitKey()
cv2.destroyAllWindows()

In [11]:
for j in range(16):
    i = sim[j].argmax()
    #p1 = pantone(img[j])
    #p2 = pantone(img[i])

    #cv2.imshow("1",p1)
    #cv2.imshow("2",p2)
    cv2.imshow("1v",img[j].numpy())
    cv2.imshow("2v",img[i].numpy())
    K = cv2.waitKey()
    if K == 27:
        cv2.destroyAllWindows()
        break
    if K == 100:
        print("FUNZIONE")
        cv2.destroyAllWindows()
        break
    cv2.destroyAllWindows()

# 1 vs ALL

In [12]:
train_loader = DataLoader(
        train_dataset,
        batch_size=64,
        shuffle=False,
        # num_workers=config.NUM_WORKERS,
    )

In [160]:
all_vec = None

for i, (h, img) in enumerate(tqdm(train_loader)):
    if all_vec is None:
        all_vec = h.clone()
    else:
        all_vec = torch.cat([all_vec,h],0)

100%|██████████| 304/304 [17:58<00:00,  3.55s/it]


In [161]:
df = pd.DataFrame(all_vec.numpy())
df.to_csv("dataset/vectorized.csv")

In [64]:
sdf = df.astype(pd.SparseDtype("float", 0.0))
sdf.to_csv("dataset/vectorized.csv")

In [66]:
print('dense : {:0.2f} MB'.format(df.memory_usage().sum() / 1e6))
print('sparse : {:0.2f} MB'.format(sdf.memory_usage().sum() / 1e6))


dense : 6.16 MB
sparse : 1.27 MB


In [16]:
df = pd.read_csv("dataset/vectorized.csv",index_col=0).values
df.shape

(19453, 512)

In [29]:
i = 2 #ispanico
i = 1 #nordeuropeo
i = 3000 #italiano abbronzato
i = 3 #euroasiatico
i = 5 #bella palette
i = 10 #centro america
i = 300 #carnagione chiara
i = 349 #afroamericano
i = 971 #esteuropeo
i = 1245 #nord africa
i = 200 #afroamericano
i = 500 #italiano medio
item = df[i]
item.shape

(512,)

In [30]:
D = (df**2).sum(1) * (item**2).sum()
sim = (df * item).sum(1) / D**0.5

In [31]:
best_idx = sim.argsort()[::-1][1:]
best_idx

array([14172,  1614,  2717, ..., 16836, 16272,  7705], dtype=int64)

In [32]:
img = train_dataset[i][1]

for j in best_idx.tolist()[:10]:
    img = np.concatenate([img, train_dataset[j][1]], 1)

img.shape

(256, 2112, 3)

In [33]:
cv2.imshow("", img)
cv2.waitKey()
cv2.destroyAllWindows()